### 实验12：Python关联分析

##### 1．掌握利用Python实现关联分析的方法

#### 二：内容和方法
使用Apriori算法建立关联规则。步骤如下：<br>
1.导入Income数据，查看Income的前5个事项，了解数据集的项集。<br>
2.查看Income中各个项的支持度，并单独查看项”age=14-34”和项”sex=male”的支持度，查看支持度最大的前10个项。<br>
3.以最小支持度0.1、最小置信度0.5建立Apriori关联规则，得到的关联规则记为rule1。<br>
4.以最小支持度0.1、最小置信度0.6建立Apriori关联规则，得到的关联规则记为rule2。<br>
5.以最小支持度0.2、最小置信度0.5建立Apriori关联规则，得到的关联规则记为rule3。<br>
比较三个关联规则的结论。

In [57]:
#-*- coding: utf-8 -*-
from __future__ import print_function
import pandas as pd

#自定义连接函数，用于实现L_{k-1}到C_k的连接
def connect_string(x, ms):
  x = list(map(lambda i:sorted(i.split(ms)), x))
  l = len(x[0])
  r = []
  for i in range(len(x)):
    for j in range(i,len(x)):
      if x[i][:l-1] == x[j][:l-1] and x[i][l-1] != x[j][l-1]:
        r.append(x[i][:l-1]+sorted([x[j][l-1],x[i][l-1]]))
  return r

#寻找关联规则的函数
def find_rule(d, support, confidence, ms = u'--'):
  result = pd.DataFrame(index=['support', 'confidence']) #定义输出结果
  
  support_series = 1.0*d.sum()/len(d) #支持度序列
  column = list(support_series[support_series > support].index) #初步根据支持度筛选
  k = 0
  
  while len(column) > 1:
    k = k+1
    print(u'\n正在进行第%s次搜索...' %k)
    column = connect_string(column, ms)
    print(u'数目：%s...' %len(column))
    sf = lambda i: d[i].prod(axis=1, numeric_only = True) #新一批支持度的计算函数
    
    #创建连接数据，这一步耗时、耗内存最严重。当数据集较大时，可以考虑并行运算优化。
    d_2 = pd.DataFrame(list(map(sf,column)), index = [ms.join(i) for i in column]).T
    
    support_series_2 = 1.0*d_2[[ms.join(i) for i in column]].sum()/len(d) #计算连接后的支持度
    column = list(support_series_2[support_series_2 > support].index) #新一轮支持度筛选
    support_series = support_series.append(support_series_2)
    column2 = []
    
    for i in column: #遍历可能的推理，如{A,B,C}究竟是A+B-->C还是B+C-->A还是C+A-->B？
      i = i.split(ms)
      for j in range(len(i)):
        column2.append(i[:j]+i[j+1:]+i[j:j+1])
    
    cofidence_series = pd.Series(index=[ms.join(i) for i in column2]) #定义置信度序列
 
    for i in column2: #计算置信度序列
      cofidence_series[ms.join(i)] = support_series[ms.join(sorted(i))]/support_series[ms.join(i[:len(i)-1])]
    
    for i in cofidence_series[cofidence_series > confidence].index: #置信度筛选
      result[i] = 0.0
      result[i]['confidence'] = cofidence_series[i]
      result[i]['support'] = support_series[ms.join(sorted(i.split(ms)))]
  
  result = result.T.sort_values(['confidence','support'], ascending = False) #结果整理，输出
  print(u'\n结果为：')
  print(result)
  
  return result

In [58]:
# 1.导入Income数据，查看Income的前5个事项，了解数据集的项集。
from __future__ import print_function
import pandas as pd

inputfile = 'D:/root/Desktop/各科/kk/12/9_Income.xlsm'
outputfile = 'D:/root/Desktop/各科/kk/12/apriori_rules.xls' #结果文件
data = pd.read_excel(inputfile).iloc[:,1:]
data.head()

,"income=$0-$40,000","income=$40,000+",sex=male,sex=female,marital status=married,marital status=cohabitation,marital status=divorced,marital status=widowed,marital status=single,age=14-34,...,ethnic classification=asian,ethnic classification=black,ethnic classification=east indian,ethnic classification=hispanic,ethnic classification=pacific islander,ethnic classification=white,ethnic classification=other,language in home=english,language in home=spanish,language in home=other
0,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,0,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,0
2,1,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,1,0,1,0,0
3,1,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,1,0,1,0,0
4,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [59]:
# 2.查看Income中各个项的支持度，并单独查看项”age=14-34”和项”sex=male”的支持度，查看支持度最大的前10个项。
# 1）查看Income中各个项的支持度
for i in data:
    print(i,'支持度为：',data[i].mean())

income=$0-$40,000 支持度为： 0.622454915648633
income=$40,000+ 支持度为： 0.3775450843513671
sex=male 支持度为： 0.44604421175101805
sex=female 支持度为： 0.553955788248982
marital status=married 支持度为： 0.3856893542757417
marital status=cohabitation 支持度为： 0.07795229784758581
marital status=divorced 支持度为： 0.09787667248400232
marital status=widowed 支持度为： 0.029377545084351368
marital status=single 支持度为： 0.40910413030831877
age=14-34 支持度为： 0.585369400814427
age=35+ 支持度为： 0.414630599185573
education=no college graduate 支持度为： 0.7052065154159395
education=college graduate 支持度为： 0.2947934845840605
occupation=professional/managerial 支持度为： 0.3392961023851076
occupation=sales 支持度为： 0.0897324025596277
occupation=laborer 支持度为： 0.08580570098894706
occupation=clerical/service 支持度为： 0.1212914485165794
occupation=homemaker 支持度为： 0.07329842931937172
occupation=student 支持度为： 0.16404886561954624
occupation=military 支持度为： 0.02036067481093659
occupation=retired 支持度为： 0.07097149505526469
occupation=unemployed 支持度为： 0.03519488074

In [61]:
# 2)并单独查看项”age=14-34”和项”sex=male”的支持度
print('age=14-13的支持度为：',data['age=14-34'].mean())
print('sex=male的支持度为：',data['sex=male'].mean())
# 3）查看支持度最大的前10个项。
sorted({i:data[i].mean() for i in data}.items(),key=lambda x:x[1],reverse=True)[:10]

age=14-13的支持度为： 0.585369400814427
sex=male的支持度为： 0.44604421175101805


[('language in home=english', 0.9128853984874927),
 ('education=no college graduate', 0.7052065154159395),
 ('number in household=1', 0.691826643397324),
 ('ethnic classification=white', 0.6697207678883071),
 ('years in bay area=10+', 0.6465968586387435),
 ('income=$0-$40,000', 0.622454915648633),
 ('number of children=0', 0.6218731820826061),
 ('dual incomes=not married', 0.5983129726585223),
 ('type of home=house', 0.5965677719604421),
 ('age=14-34', 0.585369400814427)]

In [54]:
# 3.以最小支持度0.1、最小置信度0.5建立Apriori关联规则，得到的关联规则记为rule1。
support1=0.1
confidence1=0.5
ms='---'
rule1=find_rule(data.iloc[:,1:],support=support1,confidence=confidence1,ms=ms)
rule1


正在进行第1次搜索...
数目：406...

正在进行第2次搜索...
数目：1801...

正在进行第3次搜索...
数目：3342...

正在进行第4次搜索...
数目：3313...

正在进行第5次搜索...
数目：1472...

正在进行第6次搜索...
数目：234...

正在进行第7次搜索...
数目：6...

结果为：
                                                     support  confidence
marital status=single---dual incomes=not married    0.409104    1.000000
age=14-34---marital status=single---dual income...  0.365474    1.000000
language in home=english---marital status=singl...  0.364747    1.000000
age=14-34---language in home=english---marital ...  0.324026    1.000000
education=no college graduate---marital status=...  0.320681    1.000000
...                                                      ...         ...
education=no college graduate---ethnic classifi...  0.124200    0.500293
number of children=0---sex=female---age=35+         0.160704    0.500226
householder status=rent---number in household=1...  0.170739    0.500213
age=14-34---marital status=single---sex=male        0.182810    0.500199
age=14-34---dual inco

,support,confidence
marital status=single---dual incomes=not married,0.409104,1.000000
age=14-34---marital status=single---dual incomes=not married,0.365474,1.000000
language in home=english---marital status=single---dual incomes=not married,0.364747,1.000000
age=14-34---language in home=english---marital status=single---dual incomes=not married,0.324026,1.000000
education=no college graduate---marital status=single---dual incomes=not married,0.320681,1.000000
...,...,...
education=no college graduate---ethnic classification=white---language in home=english---number in household=1---number of children=0---type of home=house,0.124200,0.500293
number of children=0---sex=female---age=35+,0.160704,0.500226
householder status=rent---number in household=1---years in bay area=1-9,0.170739,0.500213
age=14-34---marital status=single---sex=male,0.182810,0.500199


In [55]:
# 4.以最小支持度0.1、最小置信度0.6建立Apriori关联规则，得到的关联规则记为rule2。
support2=0.1
confidence2=0.6
rule2=find_rule(data.iloc[:,1:],support=support2,confidence=confidence2,ms=ms)
rule2


正在进行第1次搜索...
数目：406...

正在进行第2次搜索...
数目：1801...

正在进行第3次搜索...
数目：3342...

正在进行第4次搜索...
数目：3313...

正在进行第5次搜索...
数目：1472...

正在进行第6次搜索...
数目：234...

正在进行第7次搜索...
数目：6...

结果为：
                                                     support  confidence
marital status=single---dual incomes=not married    0.409104    1.000000
age=14-34---marital status=single---dual income...  0.365474    1.000000
language in home=english---marital status=singl...  0.364747    1.000000
age=14-34---language in home=english---marital ...  0.324026    1.000000
education=no college graduate---marital status=...  0.320681    1.000000
...                                                      ...         ...
marital status=married---type of home=house---y...  0.126672    0.600276
language in home=english---number in household=...  0.151542    0.600230
age=35+---marital status=married---years in bay...  0.119837    0.600146
ethnic classification=white---language in home=...  0.127254    0.600137
marital status=marrie

,support,confidence
marital status=single---dual incomes=not married,0.409104,1.000000
age=14-34---marital status=single---dual incomes=not married,0.365474,1.000000
language in home=english---marital status=single---dual incomes=not married,0.364747,1.000000
age=14-34---language in home=english---marital status=single---dual incomes=not married,0.324026,1.000000
education=no college graduate---marital status=single---dual incomes=not married,0.320681,1.000000
...,...,...
marital status=married---type of home=house---years in bay area=10+---sex=female,0.126672,0.600276
language in home=english---number in household=1---number of children=0---sex=male---years in bay area=10+,0.151542,0.600230
age=35+---marital status=married---years in bay area=10+---education=no college graduate,0.119837,0.600146
ethnic classification=white---language in home=english---number of children=1+---age=14-34,0.127254,0.600137


In [56]:
support3=0.2
confidence3=0.5
rule3=find_rule(data.iloc[:,1:],support=support3,confidence=confidence3,ms=ms)
rule3


正在进行第1次搜索...
数目：300...

正在进行第2次搜索...
数目：743...

正在进行第3次搜索...
数目：588...

正在进行第4次搜索...
数目：160...

正在进行第5次搜索...
数目：5...

结果为：
                                                     support  confidence
marital status=single---dual incomes=not married    0.409104    1.000000
age=14-34---marital status=single---dual income...  0.365474    1.000000
language in home=english---marital status=singl...  0.364747    1.000000
age=14-34---language in home=english---marital ...  0.324026    1.000000
education=no college graduate---marital status=...  0.320681    1.000000
...                                                      ...         ...
education=no college graduate---number in house...  0.226876    0.502254
age=14-34---householder status=rent                 0.293921    0.502112
ethnic classification=white---language in home=...  0.205207    0.500532
number in household=1---type of home=house          0.346277    0.500526
age=14-34---education=no college graduate---num...  0.229639    0.500317


,support,confidence
marital status=single---dual incomes=not married,0.409104,1.000000
age=14-34---marital status=single---dual incomes=not married,0.365474,1.000000
language in home=english---marital status=single---dual incomes=not married,0.364747,1.000000
age=14-34---language in home=english---marital status=single---dual incomes=not married,0.324026,1.000000
education=no college graduate---marital status=single---dual incomes=not married,0.320681,1.000000
...,...,...
education=no college graduate---number in household=1---householder status=rent,0.226876,0.502254
age=14-34---householder status=rent,0.293921,0.502112
ethnic classification=white---language in home=english---number in household=1---number of children=0---type of home=house,0.205207,0.500532
number in household=1---type of home=house,0.346277,0.500526
